Created on Thu Feb  2 14:12:22 2017

@author: wijers

takes a .npz box (region) projection from make_maps, convolves with a PSF, 
selects a region if needed

input: 
 - 2D array repesenting emission
 - angle subtended by an image pixel
 - centre on chip (pixel coordinates)
 - angle of chip long axis wrt image x-axis (radians)
output:
 - photon counts or ergs per second (as in input file) per pixel

In [1]:
import numpy as np
import astropy as asp
import astropy.convolution as cnv
import os.path

import eagle_constants_and_units as cons
#import loadnpz_and_plot as lnp
from cosmo_utils import comoving_distance_cm # loading all of make_maps means the entire read_eagle machinary too 

# working units: cgs, radians

radians = 1.
degrees = np.pi/180.
arcmin = 1./60. * degrees 
arcsec = 1./3600. * degrees

g = 1.
kg = 10.**3 * g

s = 1.
Hz = 1/s

cm = 1.
nm = 10.**-7 * cm
mum = 10**-4 * cm

erg = 1.

Jy = 10**-23 * erg/(s*cm**2) *1/Hz  



In [2]:
# Dragonfly CCD data (current)
data_version = 1
saved_kernels = 'dragonfly_kernels_dataversion%s' %str(data_version)
saved_abr_fftkernels = 'dragonfly_abr_fftkernels_dataversion%s' %str(data_version)

FOV = (2.6*degrees, 1.9*degrees)
numpix_dr = (3326,2504)
pix_size_phys = 5.4 *mum
pix_size_angle = 2.8 * arcsec
seeing_limit = 1.5*arcsec #typically 1.5--2.0 in long exposures according to the dragonfly paper

def diffraction_limit(wavelength):
    return 0.65*arcsec * wavelength/(450.*nm)
def airy_size(wavelength): #3.1 mum diamater of Airy disc; used as distance to first Airy zero
    return 3.1*mum/2. * wavelength/(450.*nm)

wavelengths = {'halpha': 656.28*nm}

def SB_photonscgs_to_ABmagarcsec(sb):
    fld = sb * cons.planck * (4*np.pi)/3600**2 # flux density: energy / (m**2 s)  /frequency (/arcmin**2)  sufrace brightness: photons / (cm**2 s) /sr       
    return -2.5*np.log10(fld/(3631.*Jy))
